## PySchools Full Report

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

In [2]:
#calculate number of schools, students, budget
total_schools = school_data["school_name"].count()
total_students = student_data["student_name"].count()
total_budget = "${:,.2f}". format(school_data["budget"].sum())

#calculate average scores
avg_math = student_data["math_score"].mean()
ang_reading = school_data_complete["reading_score"].mean()

#calculate total number of student passing
total_reading_pass = student_data[student_data['reading_score']>=70]['student_name'].count()
total_math_pass = student_data[student_data['math_score']>=70]['student_name'].count()
total_pass = student_data[(student_data['math_score']>=70) & (student_data['reading_score']>=70)]['student_name'].count()

#format percentages
p_reading_pass = "{0:.2%}".format(total_reading_pass/total_students)
p_math_pass = "{0:.2%}".format(total_math_pass/total_students)
p_total_pass = "{0:.2%}".format(total_pass/total_students)

In [3]:
#add all calculated values into a dataframe
district_summary_df = pd.DataFrame({
    "Total Schools" : [total_schools],
    "Total Students" : [total_students],
    "Total Budget" : [total_budget],
    "Average Math Score" : [avg_math],
    "Average Reading Score" : [ang_reading],
    "% Passing Math" : [p_math_pass],
    "% Passing Reading" : [p_reading_pass],
    "% Overall Passing" : [p_total_pass],   
})

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## School Summary

In [4]:
#create series for mean reading and math scores 

#copy school data complete with new index
new_sdc = school_data_complete.set_index('school_name')

#group by school_name
new_sdc = new_sdc.groupby('school_name')

#calculate series
reading_mean_s = new_sdc['reading_score'].mean()
math_mean_s = new_sdc['math_score'].mean()

In [5]:
#create series for percentages of student passing

#copy student data with new index
new_sd = student_data.set_index('school_name')

#calculate total students but school in series
total_students = pd.Series(new_sd.loc[:,'student_name']).groupby('school_name').count()

#caculate total passing students then divid by total student
total_reading_pass = pd.Series(new_sd.loc[new_sd['reading_score']>=70,'student_name']).groupby('school_name').count()
p_reading_pass_s = total_reading_pass.divide(total_students)

total_math_pass = pd.Series(new_sd.loc[new_sd['math_score']>=70,'student_name']).groupby('school_name').count()
p_math_pass_s = total_math_pass.divide(total_students)

total_overall_pass = pd.Series(new_sd.loc[(new_sd['math_score']>=70)&(new_sd['reading_score']>=70),'student_name']).groupby('school_name').count()
p_overall_pass_s = total_overall_pass.divide(total_students)

In [6]:
#zip new series together to merge with summary df later
school_summary_stats = {
    'average math score' : math_mean_s,
    'average reading score' : reading_mean_s,
    '%math pass' : p_math_pass_s,
    '%reading pass' : p_reading_pass_s,
    '%overall pass' : p_overall_pass_s,
}

school_summary_stats_df = pd.DataFrame(school_summary_stats)

In [7]:
#start summary df
school_summary_df = school_data[['school_name','type','size','budget']]

#create column for budget per student
school_summary_df['budget_per_student'] = school_summary_df['budget']/school_summary_df['size']

#add statistics df
school_summary_df = pd.merge(school_summary_df, school_summary_stats_df, on=["school_name", "school_name"])



#copy table to format and rename

school_summary_renamed_df = school_summary_df.copy()

school_summary_renamed_df = school_summary_renamed_df.set_index('school_name')
school_summary_renamed_df = school_summary_renamed_df.sort_values('school_name')

school_summary_renamed_df["budget"] = school_summary_renamed_df["budget"].map("${:,.2f}".format)
school_summary_renamed_df["budget_per_student"] = school_summary_renamed_df["budget_per_student"].map("${:,.2f}".format)
school_summary_renamed_df["average math score"] = school_summary_renamed_df["average math score"].map("{:.2f}".format)
school_summary_renamed_df["average reading score"] = school_summary_renamed_df["average reading score"].map("{:.2f}".format)
school_summary_renamed_df["%math pass"] = school_summary_renamed_df["%math pass"].map("{:.2%}".format)
school_summary_renamed_df["%reading pass"] = school_summary_renamed_df["%reading pass"].map("{:.2%}".format)
school_summary_renamed_df["%overall pass"] = school_summary_renamed_df["%overall pass"].map("{:.2%}".format)

new_names = {
    'school_name' : 'School Name',
    'type': 'School Type',
    'size': 'Total Students',
    'budget': 'Total School Budget',
    'budget_per_student' : 'Per Student Budget', 
    'average math score' : 'Average Math Score',
    'average reading score' : 'Average Reading Score', 
    '%math pass': "% Passing Math", 
    '%reading pass' :'% Passing Reading',
    '%overall pass': '% Passing Overall'
}

school_summary_renamed_df = school_summary_renamed_df.rename(columns=new_names)

school_summary_renamed_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

In [8]:
#sort descending
top_five_preformance_df = school_summary_df.sort_values("%overall pass", ascending = False).head()

#format and rename
top_five_preformance_df["budget"] = top_five_preformance_df["budget"].map("${:,.2f}".format)
top_five_preformance_df["budget_per_student"] = top_five_preformance_df["budget_per_student"].map("${:,.2f}".format)
top_five_preformance_df["average math score"] = top_five_preformance_df["average math score"].map("{:.2f}".format)
top_five_preformance_df["average reading score"] = top_five_preformance_df["average reading score"].map("{:.2f}".format)
top_five_preformance_df["%math pass"] = top_five_preformance_df["%math pass"].map("{:.2%}".format)
top_five_preformance_df["%reading pass"] = top_five_preformance_df["%reading pass"].map("{:.2%}".format)
top_five_preformance_df["%overall pass"] = top_five_preformance_df["%overall pass"].map("{:.2%}".format)

top_five_preformance_df = top_five_preformance_df.rename(columns = new_names)

top_five_preformance_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [9]:
#sort ascending
bottom_five_preformance_df = school_summary_df.sort_values("%overall pass", ascending = True).head()

#format and rename
bottom_five_preformance_df["budget"] = bottom_five_preformance_df["budget"].map("${:,.2f}".format)
bottom_five_preformance_df["budget_per_student"] = bottom_five_preformance_df["budget_per_student"].map("${:,.2f}".format)
bottom_five_preformance_df["average math score"] = bottom_five_preformance_df["average math score"].map("{:.2f}".format)
bottom_five_preformance_df["average reading score"] = bottom_five_preformance_df["average reading score"].map("{:.2f}".format)
bottom_five_preformance_df["%math pass"] = bottom_five_preformance_df["%math pass"].map("{:.2%}".format)
bottom_five_preformance_df["%reading pass"] = bottom_five_preformance_df["%reading pass"].map("{:.2%}".format)
bottom_five_preformance_df["%overall pass"] = bottom_five_preformance_df["%overall pass"].map("{:.2%}".format)

bottom_five_preformance_df = bottom_five_preformance_df.rename(columns = new_names)

bottom_five_preformance_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

In [10]:
#copy student data with new index
new_sdf = student_data.set_index('school_name')


#create series for each grade
m_ninth_s = pd.Series(new_sdf.loc[new_sdf['grade']=='9th','math_score'])
m_ninth_s = m_ninth_s.groupby('school_name').mean()

m_tenth_s = pd.Series(new_sdf.loc[new_sdf['grade']=='10th','math_score'])
m_tenth_s = m_tenth_s.groupby('school_name').mean()

m_eleventh_s = pd.Series(new_sdf.loc[new_sdf['grade']=='11th','math_score'])
m_eleventh_s = m_eleventh_s.groupby('school_name').mean()

m_twelfth_s = pd.Series(new_sdf.loc[new_sdf['grade']=='12th','math_score'])
m_twelfth_s = m_twelfth_s.groupby('school_name').mean()

#build dataframe
math_by_school = {
    '9th': m_ninth_s,
    '10th' : m_tenth_s,
    '11th' : m_eleventh_s,
    '12th' : m_twelfth_s
}

math_by_school_df = pd.DataFrame(math_by_school)

#format
math_by_school_df["9th"] = math_by_school_df["9th"].map("{:.2f}".format)
math_by_school_df["10th"] = math_by_school_df["10th"].map("{:.2f}".format)
math_by_school_df["11th"] = math_by_school_df["11th"].map("{:.2f}".format)
math_by_school_df["12th"] = math_by_school_df["12th"].map("{:.2f}".format)

math_by_school_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

In [11]:
#replicate with reading scores

new_sdf = student_data.set_index('school_name')

r_ninth_s = pd.Series(new_sdf.loc[new_sdf['grade']=='9th','reading_score'])
r_ninth_s = r_ninth_s.groupby('school_name').mean()

r_tenth_s = pd.Series(new_sdf.loc[new_sdf['grade']=='10th','reading_score'])
r_tenth_s = r_tenth_s.groupby('school_name').mean()

r_eleventh_s = pd.Series(new_sdf.loc[new_sdf['grade']=='11th','reading_score'])
r_eleventh_s = r_eleventh_s.groupby('school_name').mean()

r_twelfth_s = pd.Series(new_sdf.loc[new_sdf['grade']=='12th','reading_score'])
r_twelfth_s = r_twelfth_s.groupby('school_name').mean()

reading_by_school = {
    '9th': r_ninth_s,
    '10th' : r_tenth_s,
    '11th' : r_eleventh_s,
    '12th' : r_twelfth_s
}

reading_by_school_df = pd.DataFrame(reading_by_school)

reading_by_school_df["9th"] = reading_by_school_df["9th"].map("{:.2f}".format)
reading_by_school_df["10th"] = reading_by_school_df["10th"].map("{:.2f}".format)
reading_by_school_df["11th"] = reading_by_school_df["11th"].map("{:.2f}".format)
reading_by_school_df["12th"] = reading_by_school_df["12th"].map("{:.2f}".format)

reading_by_school_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [12]:
#extract data from school summary
school_spending_df = school_summary_df[['budget_per_student', 'average reading score',
       'average math score', '%reading pass', '%math pass', '%overall pass']]

#reset indexes
school_spending_df = school_spending_df.reset_index()
school_spending_df = school_spending_df.drop('index', axis = 1)

#create bins
bins = [0,584,629,644,680]
bin_labels = ['<$585', '$585-630','$630-645', '$645-680']
school_spending_df["Spending Ranges per Student"] = pd.cut(school_spending_df['budget_per_student'], bins, labels = bin_labels, include_lowest = True)
school_spending_df = school_spending_df.groupby('Spending Ranges per Student').mean()

#drop budget_per_student column
school_spending_df = school_spending_df.drop('budget_per_student', axis = 1)

#format and rename
school_spending_df["average math score"] = school_spending_df["average math score"].map("{:.2f}".format)
school_spending_df["average reading score"] = school_spending_df["average reading score"].map("{:.2f}".format)
school_spending_df["%math pass"] = school_spending_df["%math pass"].map("{:.2%}".format)
school_spending_df["%reading pass"] = school_spending_df["%reading pass"].map("{:.2%}".format)
school_spending_df["%overall pass"] = school_spending_df["%overall pass"].map("{:.2%}".format)

school_spending_df = school_spending_df.rename(columns = new_names)

school_spending_df

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
Spending Ranges per Student,,,,,
<$585,83.93,83.46,96.61%,93.46%,90.37%
$585-630,83.16,81.90,92.72%,87.13%,81.42%
$630-645,81.62,78.52,84.39%,73.48%,62.86%
$645-680,81.03,77.00,81.13%,66.16%,53.53%


## Scores by School Size

In [13]:
#replicate with school size

school_size_df = school_summary_df[['size', 'average reading score',
       'average math score', '%reading pass', '%math pass', '%overall pass']]
school_size_df = school_size_df.reset_index()
school_size_df = school_size_df.drop('index', axis = 1)

bins = [0,999,1999,4999]
bin_labels = ['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']

school_size_df["School Size"] = pd.cut(school_size_df['size'], bins, labels = bin_labels, include_lowest = True)
school_size_df = school_size_df.groupby('School Size').mean()
school_size_df = school_size_df.drop('size', axis = 1)

school_size_df["average math score"] = school_size_df["average math score"].map("{:.2f}".format)
school_size_df["average reading score"] = school_size_df["average reading score"].map("{:.2f}".format)
school_size_df["%math pass"] = school_size_df["%math pass"].map("{:.2%}".format)
school_size_df["%reading pass"] = school_size_df["%reading pass"].map("{:.2%}".format)
school_size_df["%overall pass"] = school_size_df["%overall pass"].map("{:.2%}".format)

school_size_df = school_size_df.rename(columns = new_names)

school_size_df

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
School Size,,,,,
Small (<1000),83.93,83.82,96.10%,93.55%,89.88%
Medium (1000-2000),83.86,83.37,96.79%,93.60%,90.62%
Large (2000-5000),81.34,77.75,82.77%,69.96%,58.29%


## Scores by School Type

In [14]:
#replicate with school size

school_type_df = school_summary_df[['type', 'average reading score',
       'average math score', '%reading pass', '%math pass', '%overall pass']]
school_type_df = school_type_df.reset_index()
school_type_df = school_type_df.drop('index', axis = 1)

school_type_df = school_type_df.groupby('type').mean()

school_type_df["average math score"] = school_type_df["average math score"].map("{:.2f}".format)
school_type_df["average reading score"] = school_type_df["average reading score"].map("{:.2f}".format)
school_type_df["%math pass"] = school_type_df["%math pass"].map("{:.2%}".format)
school_type_df["%reading pass"] = school_type_df["%reading pass"].map("{:.2%}".format)
school_type_df["%overall pass"] = school_type_df["%overall pass"].map("{:.2%}".format)


school_type_df = school_type_df.rename(columns = new_names)


school_type_df

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
type,,,,,
Charter,83.90,83.47,96.59%,93.62%,90.43%
District,80.97,76.96,80.80%,66.55%,53.67%
